# 第5章 誤差逆伝播法
## 5.4 単純なレイヤの実装
### 5.4.1 乗算レイヤの実装

In [2]:
%run buy_apple.py

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


### 5.4.2 加算レイヤの実装

In [3]:
%run buy_apple_orange.py

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 5.5 活性化関数レイヤの実装
### 5.5.1 ReLUレイヤ

In [4]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [5]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### 5.5.2 Sigmoidレイヤ

In [6]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6 Affine/Softmaxレイヤの実装
### 5.6.2 バッチ版Affineレイヤ

In [7]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [8]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [9]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = B
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### 5.6.3 Softmax-with-Lossレイヤ

In [10]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None    # Softmaxの出力
        self.t = None    # 教師データ

    def forward(self, x, t):
        self.t =t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

## 5.7 誤差逆伝播法の実装
### 5.7.3 誤差逆伝播法の勾配確認

In [12]:
%run gradient_check.py

W1 : 4.2718419570811224e-10
b1 : 2.487455895004058e-09
W2 : 5.352846063640126e-09
b2 : 1.3994450889093235e-07


### 5.7.4 誤差逆伝播法を使った学習

In [14]:
%run train_neuralnet.py

0.20848333333333333 0.2143
0.9036 0.9059
0.9240333333333334 0.9263
0.9347333333333333 0.9335
0.94245 0.9399
0.9514833333333333 0.9495
0.954 0.9523
0.9597 0.9548
0.96425 0.958
0.96605 0.9602
0.9686666666666667 0.9628
0.9701166666666666 0.9635
0.9729166666666667 0.9639
0.9749333333333333 0.9668
0.9761 0.9653
0.9757166666666667 0.9668
0.9785166666666667 0.9676
